<a href="https://colab.research.google.com/github/saraanwer15/Brain_Tumour/blob/main/data_block.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import random

import numpy as np
import torch
from skimage.io import imread
from torch.utils.data import Dataset
import argparse

In [3]:
class BrainSegmentationDataset(Dataset):
    """Brain MRI dataset for FLAIR abnormality segmentation"""

    in_channels = 3
    out_channels = 1

    def __init__(
        self,
        images_dir,
        transform=None,
        image_size=256,
        subset="train",
        random_sampling=True,
        validation_cases=10,
        seed=42,
    ):
        assert subset in ["all", "train", "validation"]

        # read images
        volumes = {}
        masks = {}
        print("reading {} images...".format(subset))
        for (dirpath, dirnames, filenames) in os.walk(images_dir):
            image_slices = []
            mask_slices = []
            for filename in sorted(
                filter(lambda f: ".tif" in f, filenames),
                key=lambda x: int(x.split(".")[-2].split("_")[4]),
            ):
                filepath = os.path.join(dirpath, filename)
                if "mask" in filename:
                    mask_slices.append(imread(filepath, as_gray=True))
                else:
                    image_slices.append(imread(filepath))
            if len(image_slices) > 0:
                patient_id = dirpath.split("/")[-1]
                volumes[patient_id] = np.array(image_slices[1:-1])
                masks[patient_id] = np.array(mask_slices[1:-1])

        self.patients = sorted(volumes)

        # select cases to subset
        if not subset == "all":
            random.seed(seed)
            validation_patients = random.sample(self.patients, k=validation_cases)
            if subset == "validation":
                self.patients = validation_patients
            else:
                self.patients = sorted(
                    list(set(self.patients).difference(validation_patients))
                )

        print("preprocessing {} volumes...".format(subset))
        # create list of tuples (volume, mask)
        self.volumes = [(volumes[k], masks[k]) for k in self.patients]

        """ print("cropping {} volumes...".format(subset))
        # crop to smallest enclosing volume
        self.volumes = [crop_sample(v) for v in self.volumes]

        print("padding {} volumes...".format(subset))
        # pad to square
        self.volumes = [pad_sample(v) for v in self.volumes]

        print("resizing {} volumes...".format(subset))
        # resize
        self.volumes = [resize_sample(v, size=image_size) for v in self.volumes]

        print("normalizing {} volumes...".format(subset))
        # normalize channel-wise
        self.volumes = [(normalize_volume(v), m) for v, m in self.volumes]

"""
        # probabilities for sampling slices based on masks
        self.slice_weights = [m.sum(axis=-1).sum(axis=-1) for v, m in self.volumes]
        self.slice_weights = [
            (s + (s.sum() * 0.1 / len(s))) / (s.sum() * 1.1) for s in self.slice_weights
        ]

        # add channel dimension to masks
        self.volumes = [(v, m[..., np.newaxis]) for (v, m) in self.volumes]

        print("done creating {} dataset".format(subset))

        # create global index for patient and slice (idx -> (p_idx, s_idx))
        num_slices = [v.shape[0] for v, m in self.volumes]
        self.patient_slice_index = list(
            zip(
                sum([[i] * num_slices[i] for i in range(len(num_slices))], []),
                sum([list(range(x)) for x in num_slices], []),
            )
        )

        self.random_sampling = random_sampling

        self.transform = transform

    def __len__(self):
        return len(self.patient_slice_index)

    def __getitem__(self, idx):
        patient = self.patient_slice_index[idx][0]
        slice_n = self.patient_slice_index[idx][1]

        if self.random_sampling:
            patient = np.random.randint(len(self.volumes))
            slice_n = np.random.choice(
                range(self.volumes[patient][0].shape[0]), p=self.slice_weights[patient]
            )

        v, m = self.volumes[patient]
        image = v[slice_n]
        mask = m[slice_n]

        if self.transform is not None:
            image, mask = self.transform((image, mask))

        # fix dimensions (C, H, W)
        image = image.transpose(2, 0, 1)
        mask = mask.transpose(2, 0, 1)

        image_tensor = torch.from_numpy(image.astype(np.float32))
        mask_tensor = torch.from_numpy(mask.astype(np.float32))

        # return tensors
        return image_tensor, mask_tensor


In [4]:
import sys
sys.argv=['']
del sys

In [6]:
parser = argparse.ArgumentParser(
      description="Training U-Net model for segmentation of brain MRI"
)
parser.add_argument(
      "--batch-size",
      type=int,
      default=16,
      help="input batch size for training (default: 16)",
)
parser.add_argument(
      "--epochs",
      type=int,
      default=100,
      help="number of epochs to train (default: 100)",
)
parser.add_argument(
      "--lr",
      type=float,
      default=0.0001,
      help="initial learning rate (default: 0.001)",
)
parser.add_argument(
      "--device",
      type=str,
      default="cuda:0",
      help="device for training (default: cuda:0)",
)
parser.add_argument(
      "--workers",
      type=int,
      default=4,
      help="number of workers for data loading (default: 4)",
)
parser.add_argument(
      "--vis-images",
      type=int,
      default=200,
      help="number of visualization images to save in log file (default: 200)",
)
parser.add_argument(
      "--vis-freq",
      type=int,
      default=10,
      help="frequency of saving images to log file (default: 10)",
)  
parser.add_argument(
      "--image-size",
      type=int,
      default=256,
      help="target input image size (default: 256)",
)
parser.add_argument(
      "--aug-scale",
      type=int,
      default=0.05,
      help="scale factor range for augmentation (default: 0.05)",
)
parser.add_argument(
      "--aug-angle",
      type=int,
      default=15,
      help="rotation angle range in degrees for augmentation (default: 15)",
)
parser.add_argument(
      "--images", type=str, default="/content/drive/My Drive/archive/kaggle_3m", help="root folder with images"
)
args = parser.parse_args()

In [7]:
def datasets(args):
    train = BrainSegmentationDataset(
        images_dir=args.images,
        subset="train",
        image_size=args.image_size,
    )
    valid = BrainSegmentationDataset(
        images_dir=args.images,
        subset="validation",
        image_size=args.image_size,
        random_sampling=False,
    )
    return train, valid

In [ ]:
device = torch.device("cpu" if not torch.cuda.is_available() else args.device)
loader_train, loader_valid = datasets(args)
loaders = {"train": loader_train, "valid": loader_valid}


reading train images...


In [ ]:
%tb
